In [1]:
import twitter
import sys

#language detection
import langid

#package for sentiment analysis
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()

# get the credentials
sys.path.insert(0, '../load')
from config import credentials

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\karar\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
api = twitter.Api(consumer_key = credentials['CONSUMER_KEY'],
                 consumer_secret = credentials['CONSUMER_SECRET'],
                 access_token_key = credentials['ACCESS_TOKEN'],
                 access_token_secret = credentials['ACCESS_SECRET'])

In [3]:
def fetch_twitter_data(sia, api, start, end, coin, size = 50):
    """
    Given sentiment analyzer, a date range and coin:
    1) Get size reddit posts about that coin sorted from highest to lowest score.
    2) Remove duplicated posts.
    3) Remove non-english posts.
    4) Get sentiment scores.
        restrict_score must be in the format >int or <int
        max value for size is 500
    Date ranges are expected in 'YYY-MM-DD' format
    """
    
    def get_english_posts(posts):
        """Given a set of posts, returns only English posts."""
        english_posts = []

        for post in posts:
            if langid.classify(post['text'])[0] == 'en':
                english_posts.append(post)
        return english_posts
                
    def get_sentiment(posts, sia):
        """Given a set of posts, runs sentiment analysis on the titles and returns the posts with the analysis."""
        for post in posts:
            post['sentiment_scores'] = sia.polarity_scores(post['text'])
        return posts
    
    # convert twitter module to object     
    res = api.GetSearch(term = coin, count = size, since = start, until = end)
    posts = []
    for status in res:
        posts.append({
            'text': status.text,
            'created:': status.created_at,
            'favorites': status.favorite_count,
            'retweets': status.retweet_count,
            'followers': status.user.followers_count
        })
        
    # only keep english posts, get sentiment analysis
    posts = get_english_posts(posts)
    posts = get_sentiment(posts, sia)
    return posts
    
    

In [4]:
fetch_twitter_data(sia, api, '2021-10-01', '2021-10-02', 'Ethereum', size = 1)

[{'text': '#AuctionUpdate Curio Cards, a set of 31 NFTs considered to be the oldest artworks on the Ethereum blockchain sold f… https://t.co/VXbjJRxVai',
  'created:': 'Fri Oct 01 14:35:11 +0000 2021',
  'favorites': 1591,
  'retweets': 591,
  'followers': 168013,
  'sentiment_scores': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}}]